In [86]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from typing import List, Tuple

In [87]:
with open('inputs/dec06s.txt', 'r') as f:
    lines = f.readlines()
current_game = [list(line.strip()) for line in lines]
current_game = np.array(current_game)
current_game[current_game == '.'] = 0
current_game[current_game == '#'] = 1
current_game[current_game == '^'] = 9
current_game = current_game.astype(float)
irow_guard, icol_guard = tuple(np.where(current_game == 9))
start_position = np.append(irow_guard, icol_guard)
current_game[irow_guard, icol_guard] = 0
print(start_position)
print(current_game)
start_direction = np.array([-1, 0])

[6 4]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]]


In [88]:
def get_new_direction(game: np.ndarray, position: np.ndarray, direction: np.ndarray) -> np.ndarray:
    if position[0] + direction[0] < 0 or position[0] + direction[0] >= game.shape[0] or position[1] + direction[1] < 0 or position[1] + direction[1] >= game.shape[1]:
        return np.zeros_like(direction)
    elif game[position[0] + direction[0], position[1] + direction[1]] == 0:
        return direction.copy()
    else:
        rotation_matrix = np.array([[0, 1], [-1, 0]])
        new_direction = np.dot(rotation_matrix, direction).copy()
        return new_direction

def run(game: np.ndarray, start_position: np.ndarray, start_direction: np.ndarray) -> Tuple[np.ndarray, np.ndarray, bool]:
    footpath = np.zeros_like(game, dtype=int)
    footpath[start_position[0], start_position[1]] = 1

    position = start_position.copy()
    direction = start_direction.copy()
    is_loop = False
    while True:
        new_direction = get_new_direction(game, position, direction)
        if np.all(new_direction == np.zeros_like(new_direction)):
            break
        position += new_direction
        direction = new_direction
        footpath[position[0], position[1]] += 1
        # print(footpath.sum(), position, new_direction, game[position[0], position[1]])
        if footpath[position[0], position[1]] == 5:
            is_loop = True
            break
    
    return footpath, position, is_loop

footpath, last_position, is_loop = run(current_game, start_position, start_direction)
print(footpath[footpath > 0].size, last_position, is_loop)

41 [9 7] False


In [89]:
start = start_position + 1j * start_direction
type(start)

numpy.ndarray

In [ ]:
def run_complex(game: np.ndarray, start_state: np.ndarray) -> None:
    state = start_state
    seen = start_state
    print(state, len(seen))
    print(state not in seen)
    while state not in seen:
        pos = state.real    
        dir = state.imag
        new_pos = (pos + dir).astype(int)
        if game[new_pos[0], new_pos[1]] == 1:
            new_dir = state.imag * 1j
            new_state = pos + new_dir*1j
        else:
            new_state = new_pos + dir*1j
        if new_state in seen:
            break
        seen.append(state)
        state = new_state
        print(state, len(seen), seen)
        print(state not in seen)
    return seen

def run_obstacles(game: np.ndarray, start: np.ndarray) -> None:
    seen = run_complex(game, start)
    print(len(seen))

run_obstacles(current_game, start)

[6.-1.j 4.+0.j] 0
True
[5.-1.j 4.+0.j] 1 [array([6.-1.j, 4.+0.j])]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [91]:
def get_possible_obstacle_positions(game: np.ndarray, position: np.ndarray) -> np.ndarray:
    possible_position = np.zeros_like(game, dtype=int)
    possible_position[game == 0] = 1
    possible_position[position[0], position[1]] = 0

    directions = np.array([[0, 1], [1, 0], [0, -1], [-1, 0], [1, 1], [1, -1], [-1, 1], [-1, -1]])
    for direction in directions:
        try:
            possible_position[position[0] + direction[0], position[1] + direction[1]] = 0
        except IndexError:
            pass
    return possible_position

possible_obstacle_positions = get_possible_obstacle_positions(current_game, guard_start)

def run2(game: np.ndarray, start_position: np.ndarray, start_direction: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    footpath, _, is_loop = run(game, start_position, start_direction)
    possible_obstacle_positions = get_possible_obstacle_positions(game, start_position)
    reudced_possible_obstacle_positions = possible_obstacle_positions.copy()
    reudced_possible_obstacle_positions[footpath == 0] = 0
    irows, icols = np.where(reudced_possible_obstacle_positions == 1)
    obstacle_is_loop = np.zeros_like(irows, dtype=bool)
    loop_size = np.zeros_like(irows)
    for iobstacle, (irow, icol) in enumerate(zip(irows, icols)):
        manipulated_game = game.copy()
        manipulated_game[irow, icol] = 1
        footpath, _, obstacle_is_loop[iobstacle] = run(manipulated_game, start_position, start_direction)
        loop_size[iobstacle] = footpath[footpath > 0].size
        print(f"{iobstacle:04d}", footpath[footpath > 0].size, obstacle_is_loop[iobstacle])
    return irows, icols, obstacle_is_loop, loop_size

irows, icols, obstacle_is_loop, loop_size = run2(current_game, guard_start, np.array([-1, 0]))

IndexError: index 43 is out of bounds for axis 0 with size 10

In [ ]:
obstacle_is_loop[obstacle_is_loop].size

1833